In [ ]:
!pip install scikit-multilearn
!pip install scikit-multilearn --upgrade

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import os
import nltk
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime
from skmultilearn.adapt import mlknn
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import sklearn.svm
from sklearn.svm import SVC

from google.colab import drive
drive.mount('/content/drive')

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

text_path='/content/drive/MyDrive/colab_SMAI/test1.txt'
df=pd.read_csv(text_path, names=['Id', 'Title','Body','Tag'],skiprows=1,nrows=1200000)
df['Tag'] = df['Tag'].fillna("")
df['Tag'] = df['Tag'].astype(str)

df['cleaned_tags']=df['Tag'].str.lower().replace('\n','')
df['cleaned_tags']=df['cleaned_tags'].apply(lambda x: str(x).encode('ascii', 'ignore').decode('ascii'))
df['cleaned_tags']=df['cleaned_tags'].str.replace('\d+', '')
df['cleaned_tags']=df['cleaned_tags'].str.replace(' +', ' ')

junk_chars = "[]{}.-"
for c in junk_chars:
  df['cleaned_tags']=df['cleaned_tags'].str.replace(c, '')

qus_list=[]
qus_with_code = 0
len_before_preprocessing = 0 
len_after_preprocessing = 0 
cnt = 0
for index,row in df.iterrows():
    title, body, tags = row["Title"], row["Body"], row["cleaned_tags"]
    if '<code>' in body:
        qus_with_code+=1
    len_before_preprocessing+=len(title) + len(body)
    body=re.sub('<code>(.*?)</code>', '', body, flags=re.MULTILINE|re.DOTALL)
    body = re.sub('<.*?>', ' ', str(body.encode('utf-8')))
    title=title.encode('utf-8')
    question=str(title)+" "+str(body)
    question=re.sub(r'[^A-Za-z]+',' ',question)
    words=word_tokenize(str(question.lower()))
    question=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
    qus_list.append(question)
    len_after_preprocessing += len(question)
    cnt = cnt + 1
    if (cnt%25000 == 0):
      print("Qstn body processed : ", cnt)

df["question"] = qus_list
avg_len_before_preprocessing=(len_before_preprocessing*1.0)/df.shape[0]
avg_len_after_preprocessing=(len_after_preprocessing*1.0)/df.shape[0]

     |████████████████████████████████| 92kB 4.9MB/s 
Requirement already up-to-date: scikit-multilearn in /usr/local/lib/python3.7/dist-packages (0.2.0)
Mounted at /content/drive
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Qstn body processed :  25000
Qstn body processed :  50000
Qstn body processed :  75000
Qstn body processed :  100000
Qstn body processed :  125000
Qstn body processed :  150000
Qstn body processed :  175000
Qstn body processed :  200000
Qstn body processed :  225000
Qstn body processed :  250000
Qstn body processed :  275000
Qstn body processed :  300000
Qstn body processed :  325000
Qstn body processed :  350000
Qstn body processed :  375000
Qstn body processed :  400000
Qstn body processed :  425000
Qstn body processed :  450000
Qstn body processed :  475000
Qstn body processed :  500000
Qstn b

In [ ]:
preprocessed_df = df[["question","cleaned_tags"]]
preprocessed_df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
preprocessed_df = preprocessed_df.dropna(subset=['question','cleaned_tags'])
print("Shape of preprocessed data :", preprocessed_df.shape)

Shape of preprocessed data : (1200000, 2)


In [ ]:
preprocessed_df = df[["question","cleaned_tags"]]
preprocessed_df = preprocessed_df[:30000]
print("Shape of preprocessed data :", preprocessed_df.shape)
preprocessed_df.drop_duplicates()
print("Shape of preprocessed data :", preprocessed_df.shape)

Shape of preprocessed data : (30000, 2)
Shape of preprocessed data : (30000, 2)


In [ ]:
def tags_to_consider(n):
    tag_i_sum = y_multilabel.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
    yn_multilabel=y_multilabel[:,sorted_tags_i[:n]]
    return yn_multilabel

def questions_covered_fn(numb):
    yn_multilabel = tags_to_consider(numb)
    x= yn_multilabel.sum(axis=1)
    return (np.count_nonzero(x==0))

vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
y_multilabel = vectorizer.fit_transform(preprocessed_df['cleaned_tags'])

questions_covered = []
total_tags=y_multilabel.shape[1]
total_qus=preprocessed_df.shape[0]
for i in range(100, total_tags, 100):
    questions_covered.append(np.round(((total_qus-questions_covered_fn(i))/total_qus)*100,3))

yx_multilabel = tags_to_consider(100)
X_train, X_test, y_train, y_test = train_test_split(preprocessed_df, yx_multilabel, test_size = 0.1,random_state = 42)
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=20000, tokenizer = lambda x: x.split(), ngram_range=(1,3))
X_train_multilabel = vectorizer.fit_transform(X_train['question'])
X_test_multilabel = vectorizer.transform(X_test['question'])

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
mlpc = MLPClassifier()
mlpc.fit(X_train_multilabel, y_train)
y_pred = mlpc.predict(X_test_multilabel)
print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))
print("Hamming loss :",metrics.hamming_loss(y_test,y_pred))

Accuracy : 0.362
Macro f1 score : 0.4503418316564851
Micro f1 scoore : 0.5083237184770066
Hamming loss : 0.009943333333333334


In [ ]:
rfc = OneVsRestClassifier(RandomForestClassifier())
rfc.fit(X_train_multilabel, y_train)
y_pred = rfc.predict(X_test_multilabel)
print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))
print("Hamming loss :",metrics.hamming_loss(y_test,y_pred))

Accuracy : 0.345
Macro f1 score : 0.14284270624545806
Micro f1 scoore : 0.3578315834583244
Hamming loss : 0.00999


In [ ]:
cparam = [.65, .7, 1]
for cval in cparam:
  print(" ------------- for : "+ str(cval)+ "---------------")
  clf_lsvc = OneVsRestClassifier(LinearSVC(C=cval))
  clf_lsvc.fit(X_train_multilabel, y_train)
  y_pred = clf_lsvc.predict(X_test_multilabel)
  print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
  print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
  print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))
  print("Hamming loss :",metrics.hamming_loss(y_test,y_pred))

 ------------- for : 0.65---------------
Accuracy : 0.393
Macro f1 score : 0.4071852374590177
Micro f1 scoore : 0.4921465968586387
Hamming loss : 0.009053333333333333
 ------------- for : 0.7---------------
Accuracy : 0.392
Macro f1 score : 0.40916113215014344
Micro f1 scoore : 0.49357422238778176
Hamming loss : 0.009063333333333333
 ------------- for : 1---------------
Accuracy : 0.39166666666666666
Macro f1 score : 0.42203608028576667
Micro f1 scoore : 0.5041924899744804
Hamming loss : 0.009066666666666667


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train_multilabel)
X_train_multilabel = scaling.transform(X_train_multilabel)
X_test_multilabel = scaling.transform(X_test_multilabel)

TypeError: ignored

In [ ]:
model = OneVsRestClassifier(svm.SVC(kernel='poly', degree=3))
model.fit(X_train_multilabel, y_train)
y_pred = model.predict(X_test_multilabel)
print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))
print("Hamming loss :",metrics.hamming_loss(y_test,y_pred))